# Code EEMRIO by Martínez-Hernando


This information is necessary in Python, for activate the funtions 
If they are not installed is necessary to do it, write pip install XXXXX. If pip doesn't work put pip3, or pip3.11 (search on internet)

Activate generical information

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from scipy import io, integrate, linalg, signal #Google told me, I don´t know what is it
from scipy.sparse.linalg import cg, eigs #Google told me, I don´t know what is it

To create list

In [2]:
from typing import List, Tuple, Dict, Callable, Iterable, Union

 # To use pymrio

In [3]:
import pymrio as mr

Declare new IO system

In [4]:
exio3318=mr.IOSystem()

## Importing EXIOBASE 3.3.18 data
We will only import:
- $Z$, the intermediate consumption,
- $Y$, the final demand,
- $F$ and $F_Y$, the environmental extensions, only for air emissions.

In [5]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (Intermediate products)

In [6]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.head() #To see the first lines

AU  \
                                               Cultivation of paddy rice   
                                                                   i01.a   
                                                                  A_PARI   
AU Paddy rice              p01.a C_PARI tonnes                       0.0   
   Wheat                   p01.b C_WHEA tonnes                       0.0   
   Cereal grains nec       p01.c C_OCER tonnes                       0.0   
   Vegetables, fruit, nuts p01.d C_FVEG tonnes                       0.0   
   Oil seeds               p01.e C_OILS tonnes                       0.0   

                                                                     \
                                               Cultivation of wheat   
                                                              i01.b   
                                                             A_WHEA   
AU Paddy rice              p01.a C_PARI tonnes                  0.0   
   Wheat                   p01.b C_WHEA tonnes                  0.0   
   Cereal grains nec       p01.c C_OCER tonnes                  0.0   
   Vegetables, fruit, nuts p01.d C_FVEG tonnes                  0.0   
   Oil seeds               p01.e C_OILS tonnes                  0.0   

                                                                                 \
                                               Cultivation of cereal grains nec   
                                                                          i01.c   
                                                                         A_OCER   
AU Paddy rice              p01.a C_PARI tonnes                          0.00000   
   Wheat                   p01.b C_WHEA tonnes                          0.00000   
   Cereal grains nec       p01.c C_OCER tonnes                      70942.96547   
   Vegetables, fruit, nuts p01.d C_FVEG tonnes                          0.00000   
   Oil seeds               p01.e C_OILS tonnes                          0.00000   

                                                                                       \
                                               Cultivation of vegetables, fruit, nuts   
                                                                                i01.d   
                                                                               A_FVEG   
AU Paddy rice              p01.a C_PARI tonnes                                0.00000   
   Wheat                   p01.b C_WHEA tonnes                                0.00000   
   Cereal grains nec       p01.c C_OCER tonnes                                0.00000   
   Vegetables, fruit, nuts p01.d C_FVEG tonnes                            18266.75262   
   Oil seeds               p01.e C_OILS tonnes                                0.00000   

                                                                         \
                                               Cultivation of oil seeds   
                                                                  i01.e   
                                                                 A_OILS   
AU Paddy rice              p01.a C_PARI tonnes                      0.0   
   Wheat                   p01.b C_WHEA tonnes                      0.0   
   Cereal grains nec       p01.c C_OCER tonnes                      0.0   
   Vegetables, fruit, nuts p01.d C_FVEG tonnes                      0.0   
   Oil seeds               p01.e C_OILS tonnes                      0.0   

                                                                                      \
                                               Cultivation of sugar cane, sugar beet   
                                                                               i01.f   
                                                                              A_SUGB   
AU Paddy rice              p01.a C_PARI tonnes                                   0.0   
   Wheat                   p01.b C_WHEA tonnes                                 

In [7]:
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
Z_reindexed.head()


region                                 AU                          \
sector                         Paddy rice Wheat Cereal grains nec   
region sector                                                       
AU     Paddy rice                     0.0   0.0           0.00000   
       Wheat                          0.0   0.0           0.00000   
       Cereal grains nec              0.0   0.0       70942.96547   
       Vegetables, fruit, nuts        0.0   0.0           0.00000   
       Oil seeds                      0.0   0.0           0.00000   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                              0.00000       0.0   
       Wheat                                   0.00000       0.0   
       Cereal grains nec                       0.00000       0.0   
       Vegetables, fruit, nuts             18266.75262       0.0   
       Oil seeds                               0.00000       0.0   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                                 0.0               0.00   
       Wheat                                      0.0               0.00   
       Cereal grains nec                          0.0               0.00   
       Vegetables, fruit, nuts                    0.0               0.00   
       Oil seeds                                  0.0        -1206065.87   

region                                                                 ...  \
sector                         Crops nec        Cattle           Pigs  ...   
region sector                                                          ...   
AU     Paddy rice                    0.0  0.000000e+00       0.000000  ...   
       Wheat                         0.0  6.791574e+05  156083.893300  ...   
       Cereal grains nec             0.0  2.277000e+06  823314.442100  ...   
       Vegetables, fruit, nuts       0.0  1.552051e+03     255.322381  ...   
       Oil seeds                     0.0  0.000000e+00       0.000000  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                                0.0   
       Wheat                                                     0.0   
       Cereal grains nec                                         0.0   
       Vegetables, fruit, nuts                                   0.0   
       Oil seeds                                                 0.0   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                             0.0    
       Wheat                                                                  0.0    
       Cereal grains nec                                                      0.0    
       Vegetables, fruit, nuts                                

In [8]:
exio3318.Z=Z_reindexed

Y (Final demand)

In [9]:
Y_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_FD.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])

Y_reindexed=Y_raw.droplevel([2,3], axis=1).copy() #To remove useless columns
Y_reindexed.index=Z_reindexed.index
Y_reindexed.columns.names=["region", "sector"] #To name the columns
exio3318.Y=Y_reindexed

In [10]:
exio3318.Y.head()

region                                                                  AU  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                             0.000000e+00   
       Wheat                                                  0.000000e+00   
       Cereal grains nec                                      0.000000e+00   
       Vegetables, fruit, nuts                                1.255628e+06   
       Oil seeds                                              7.501443e+02   

region                                                                                                               \
sector                         Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                         
AU     Paddy rice                                                        0.000000                                     
       Wheat                                                        317232.259200                                     
       Cereal grains nec                                             20796.255720                                     
       Vegetables, fruit, nuts                                        9066.727501                                     
       Oil seeds                                                       414.340678                                     

region                                                                      \
sector                         Final consumption expenditure by government   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                 635980.576100   
       Cereal grains nec                                      43596.865240   
       Vegetables, fruit, nuts                                13660.087220   
       Oil seeds                                                402.405072   

region                                                        \
sector                         Gross fixed capital formation   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                      \
sector                         Changes in inventories Changes in valuables   
region sector                                                                
AU     Paddy rice                             0.00000                    0   
       Wheat                                  0.00000                    0   
       Cereal grains nec                      0.00000                    0   
       Vegetables, fruit, nuts            25217.20699                    0   
       Oil seeds                          11344.59956                    0   

region                                                                  AT  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                      0.000000   
       Cereal grains nec                                          0.645819   
       Vegetables, fruit, nuts                                  262.282173   
       Oil seeds                                                  0.000000   

region                            

Test that the core elements are well-parsed

In [11]:
exio3318.calc_system() #To calculate the rest of matrix
exio3318.A.head() #To show A (technical coefficients)
exio3318.L.head() #To show L (Leontief matrix I-A)

region                                 AU                              \
sector                         Paddy rice     Wheat Cereal grains nec   
region sector                                                           
AU     Paddy rice                1.000009  0.000008          0.000006   
       Wheat                     0.000216  1.000193          0.000137   
       Cereal grains nec         0.000088  0.000081          1.007140   
       Vegetables, fruit, nuts   0.000020  0.000019          0.000012   
       Oil seeds                 0.000009  0.000010          0.000024   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                             0.000049  0.000027   
       Wheat                                  0.001163  0.000679   
       Cereal grains nec                      0.000583  0.000366   
       Vegetables, fruit, nuts                1.005752  0.000030   
       Oil seeds                              0.000144  1.000765   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                            0.000004           0.000104   
       Wheat                                 0.000125           0.002369   
       Cereal grains nec                     0.000041           0.000951   
       Vegetables, fruit, nuts               0.000009           0.000248   
       Oil seeds                             0.000003          -1.552196   

region                                                        ...  \
sector                         Crops nec    Cattle      Pigs  ...   
region sector                                                 ...   
AU     Paddy rice               0.001503  0.005874  0.000821  ...   
       Wheat                    0.062994  0.628912  0.714555  ...   
       Cereal grains nec        0.019043  2.059080  3.760873  ...   
       Vegetables, fruit, nuts  0.003652  0.065239  0.116988  ...   
       Oil seeds                0.003211  0.068144  0.142334  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                               3.028264e-08   
       Wheat                                    2.976197e-06   
       Cereal grains nec                        3.354884e-06   
       Vegetables, fruit, nuts                  1.959977e-07   
       Oil seeds                               -5.969246e-06   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                       1.159917e-08   
       Wheat                                            3.506702e-06   
       Cereal grains nec                                2.519620e-07   
       Vegetables, fruit, nuts                          1.428908e-07   
       Oil seeds                                       -1.526406e-06   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                    2.102449e-09    
       Wheat                                                         2.009846e-07    
       Cereal grains nec                                             5.861412e-08    
       Vegetables, fruit, nuts                                       1.290512e-08    
       Oil seeds

F and F_Y (Extensions, total emissions per activity)

In [12]:
exio3318.emissions=mr.Extension("emissions")
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns

F_Y_raw=pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of final demand
                      sheet_name="Emiss_FD", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_Y_raw.columns=Y_reindexed.columns

F_raw.head()
F_Y_raw.head()
exio3318.emissions.F=F_raw
exio3318.emissions.F_Y=F_Y_raw

In [13]:
F_Y_raw.head()

region                                                                     AU  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                4.760370e+07   
N2O                    tonnes air                                9.248053e+02   
CH4                    tonnes air                                4.907784e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                           0.000000                                     
N2O                    tonnes air                                           0.000000                                     
CH4                    tonnes air                                         200.749203                                     
HFCs                   tonnes air                                           0.000000                                     
PFCs                   tonnes air                                           0.000000                                     

region                                                                         \
sector                            Final consumption expenditure by government   
Carbon dioxide, fossil tonnes air                                    0.000000   
N2O                    tonnes air                                    0.000000   
CH4                    tonnes air                                  343.422972   
HFCs                   tonnes air                                    0.000000   
PFCs                   tonnes air                                    0.000000   

region                                                           \
sector                            Gross fixed capital formation   
Carbon dioxide, fossil tonnes air                             0   
N2O                    tonnes air                             0   
CH4                    tonnes air                             0   
HFCs                   tonnes air                             0   
PFCs                   tonnes air                             0   

region                                                                         \
sector                            Changes in inventories Changes in valuables   
Carbon dioxide, fossil tonnes air                      0                    0   
N2O                    tonnes air                      0                    0   
CH4                    tonnes air                      0                    0   
HFCs                   tonnes air                      0                    0   
PFCs                   tonnes air                      0                    0   

region                                                                     AT  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                2.287570e+07   
N2O                    tonnes air                                6.054108e+02   
CH4                    tonnes air                                1.750280e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                                  0                                     
N2O                    tonnes air    

To do again the calculations

In [16]:
exio3318.reset_all_full() #reset
exio3318.calc_all() #calculation of the rest of matrix again

c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:984: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  self.D_cba.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:993: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` withou

# Information calculated by pymrio

Total production vector (hybrid units)

In [17]:
exio3318.x

indout
region sector                                                         
AU     Paddy rice                                         6.147906e+05
       Wheat                                              2.329856e+07
       Cereal grains nec                                  1.009107e+07
       Vegetables, fruit, nuts                            3.256927e+06
       Oil seeds                                          4.010747e+05
...                                                                ...
WM     Membership organisation services n.e.c. (91)       1.549903e+04
       Recreational, cultural and sporting services (92)  3.103584e+04
       Other services (93)                                2.187649e+04
       Private households with employed persons (95)      5.306091e+03
       Extra-territorial organizations and bodies         0.000000e+00

[7872 rows x 1 columns]

This are the TOTAL emissions (in exiobase units, most tonnes) per activity in 2011, consumption based

In [18]:
exio3318.emissions.D_cba 

region                                        AU                 \
sector                                Paddy rice          Wheat   
Carbon dioxide, fossil   tonnes air          0.0  186012.172009   
N2O                      tonnes air          0.0     480.028912   
CH4                      tonnes air          0.0     592.364744   
HFCs                     tonnes air          0.0       0.000000   
PFCs                     tonnes air          0.0       0.000000   
...                                          ...            ...   
Ni                       tonnes soil         0.0       0.000000   
C                        tonnes air          0.0       0.000000   
other emissions          tonnes undef        0.0    4205.077109   
Carbon dioxide, biogenic tonnes air          0.0    3223.945251   
Other emissions nec      tonnes air          0.0       0.000000   

region                                                   \
sector                                Cereal grains nec   
Carbon dioxide, fossil   tonnes air         9441.764780   
N2O                      tonnes air           31.246322   
CH4                      tonnes air           33.424020   
HFCs                     tonnes air            0.000000   
PFCs                     tonnes air            0.000000   
...                                                 ...   
Ni                       tonnes soil           0.000000   
C                        tonnes air            0.000000   
other emissions          tonnes undef        401.052921   
Carbon dioxide, biogenic tonnes air          200.760505   
Other emissions nec      tonnes air            0.000000   

region                                                                      \
sector                                Vegetables, fruit, nuts    Oil seeds   
Carbon dioxide, fossil   tonnes air              1.380421e+06  7727.839974   
N2O                      tonnes air              4.035820e+03    16.771654   
CH4                      tonnes air              5.447565e+03    34.168751   
HFCs                     tonnes air              0.000000e+00     0.000000   
PFCs                     tonnes air              0.000000e+00     0.000000   
...                                                       ...          ...   
Ni                       tonnes soil             0.000000e+00     0.000000   
C                        tonnes air              0.000000e+00     0.000000   
other emissions          tonnes undef            5.285416e+04  1372.810482   
Carbon dioxide, biogenic tonnes air              3.714176e+04   202.065860   
Other emissions nec      tonnes air              0.000000e+00     0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air             19996.958208   
N2O                      tonnes air                58.037292   
CH4                      tonnes air                66.083534   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef             522.415175   
Carbon dioxide, biogenic tonnes air               473.701020   
Other emissions nec      tonnes air                 0.000000   

region                                                                        \
sector                                Plant-based fibers    Crops nec Cattle   
Carbon dioxide, fossil   tonnes air                  0.0  8152.776632    0.0   
N2O                      tonnes air                  0.0    12.521482    0.0   
CH4                      tonnes air                  0.0    35.498277    0.0   
HFCs                     tonnes air                  0.0     0.000000    0.0   
PFCs                     tonnes 

This are the TOTAL emissions (in exiobase units, most tonnes) per activity in 2011, production based

In [19]:
exio3318.emissions.D_pba #D_pba should be the same as F
#exio3318.emissions.D_pba.xs("Nickel ores and concentrates", level="sector", axis=1) #To look for a sector in different countries/region

region                                           AU                \
sector                                   Paddy rice         Wheat   
Carbon dioxide, fossil   tonnes air    37613.496514  1.941703e+06   
N2O                      tonnes air       44.090364  9.484618e+03   
CH4                      tonnes air    26526.840161  1.636344e+02   
HFCs                     tonnes air        0.000000  0.000000e+00   
PFCs                     tonnes air        0.000000  0.000000e+00   
...                                             ...           ...   
Ni                       tonnes soil       0.000000  0.000000e+00   
C                        tonnes air        0.000000  0.000000e+00   
other emissions          tonnes undef    441.215838  3.707975e+04   
Carbon dioxide, biogenic tonnes air       95.599361  5.275332e+03   
Other emissions nec      tonnes air        0.000000  0.000000e+00   

region                                                   \
sector                                Cereal grains nec   
Carbon dioxide, fossil   tonnes air       490896.672482   
N2O                      tonnes air         3883.950687   
CH4                      tonnes air          137.838535   
HFCs                     tonnes air            0.000000   
PFCs                     tonnes air            0.000000   
...                                                 ...   
Ni                       tonnes soil           0.000000   
C                        tonnes air            0.000000   
other emissions          tonnes undef      35398.867081   
Carbon dioxide, biogenic tonnes air         1213.369876   
Other emissions nec      tonnes air            0.000000   

region                                                                       \
sector                                Vegetables, fruit, nuts     Oil seeds   
Carbon dioxide, fossil   tonnes air             245167.152043  82995.085115   
N2O                      tonnes air               8398.870869    463.165908   
CH4                      tonnes air                 66.854787     20.549471   
HFCs                     tonnes air                  0.000000      0.000000   
PFCs                     tonnes air                  0.000000      0.000000   
...                                                       ...           ...   
Ni                       tonnes soil                 0.000000      0.000000   
C                        tonnes air                  0.000000      0.000000   
other emissions          tonnes undef            60146.077462  37620.644208   
Carbon dioxide, biogenic tonnes air                566.432429    186.354150   
Other emissions nec      tonnes air                  0.000000      0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air            173077.023726   
N2O                      tonnes air              1161.219659   
CH4                      tonnes air                22.101628   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef            3348.515947   
Carbon dioxide, biogenic tonnes air               217.376806   
Other emissions nec      tonnes air                 0.000000   

region                                                                 \
sector                                Plant-based fibers    Crops nec   
Carbon dioxide, fossil   tonnes air        151797.175764  4103.100931   
N2O                      tonnes air          1166.613596     2.362144   
CH4                      tonnes air            12.585492     0.422317   
HFCs                     tonnes air             0.000000     0.000000   
PFCs                     tonnes air

This are the TOTAL emissions (in exiobase units, most tonnes) per activity in 2011, exported and imported

In [20]:
exio3318.emissions.D_exp
exio3318.emissions.D_imp

region                                        AU                \
sector                                Paddy rice         Wheat   
Carbon dioxide, fossil   tonnes air          0.0  49366.352417   
N2O                      tonnes air          0.0     86.667189   
CH4                      tonnes air          0.0    340.904373   
HFCs                     tonnes air          0.0      0.000000   
PFCs                     tonnes air          0.0      0.000000   
...                                          ...           ...   
Ni                       tonnes soil         0.0      0.000000   
C                        tonnes air          0.0      0.000000   
other emissions          tonnes undef        0.0   1870.989952   
Carbon dioxide, biogenic tonnes air          0.0   2087.202519   
Other emissions nec      tonnes air          0.0      0.000000   

region                                                   \
sector                                Cereal grains nec   
Carbon dioxide, fossil   tonnes air         3372.654760   
N2O                      tonnes air            5.956867   
CH4                      tonnes air           19.372335   
HFCs                     tonnes air            0.000000   
PFCs                     tonnes air            0.000000   
...                                                 ...   
Ni                       tonnes soil           0.000000   
C                        tonnes air            0.000000   
other emissions          tonnes undef        117.344805   
Carbon dioxide, biogenic tonnes air          120.458393   
Other emissions nec      tonnes air            0.000000   

region                                                                      \
sector                                Vegetables, fruit, nuts    Oil seeds   
Carbon dioxide, fossil   tonnes air             569741.835088  1290.571164   
N2O                      tonnes air                612.900903     1.647491   
CH4                      tonnes air               2827.190428    16.552180   
HFCs                     tonnes air                  0.000000     0.000000   
PFCs                     tonnes air                  0.000000     0.000000   
...                                                       ...          ...   
Ni                       tonnes soil                 0.000000     0.000000   
C                        tonnes air                  0.000000     0.000000   
other emissions          tonnes undef            18799.063054    90.959411   
Carbon dioxide, biogenic tonnes air              23862.922546   131.731474   
Other emissions nec      tonnes air                  0.000000     0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air              7320.149795   
N2O                      tonnes air                10.534165   
CH4                      tonnes air                45.604178   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef             271.614852   
Carbon dioxide, biogenic tonnes air               311.878616   
Other emissions nec      tonnes air                 0.000000   

region                                                                        \
sector                                Plant-based fibers    Crops nec Cattle   
Carbon dioxide, fossil   tonnes air                  0.0  7647.432540    0.0   
N2O                      tonnes air                  0.0    12.419241    0.0   
CH4                      tonnes air                  0.0    33.720955    0.0   
HFCs                     tonnes air                  0.0     0.000000    0.0   
PFCs                     tonnes air          

This are the UNITARY emissions direct per activity and per functional unit (F/X). Called S direct emissions

In [21]:
exio3318.emissions.S
#exio3318.emissions.S.loc["Carbon dioxide, fossil", ("AU","Nickel ores and concentrates")] #To look for an specific activity

region                                        AU                              \
sector                                Paddy rice     Wheat Cereal grains nec   
Carbon dioxide, fossil   tonnes air     0.061182  0.083341          0.048647   
N2O                      tonnes air     0.000072  0.000407          0.000385   
CH4                      tonnes air     0.043148  0.000007          0.000014   
HFCs                     tonnes air     0.000000  0.000000          0.000000   
PFCs                     tonnes air     0.000000  0.000000          0.000000   
...                                          ...       ...               ...   
Ni                       tonnes soil    0.000000  0.000000          0.000000   
C                        tonnes air     0.000000  0.000000          0.000000   
other emissions          tonnes undef   0.000718  0.001592          0.003508   
Carbon dioxide, biogenic tonnes air     0.000156  0.000226          0.000120   
Other emissions nec      tonnes air     0.000000  0.000000          0.000000   

region                                                                   \
sector                                Vegetables, fruit, nuts Oil seeds   
Carbon dioxide, fossil   tonnes air                  0.075276  0.207100   
N2O                      tonnes air                  0.002579  0.001156   
CH4                      tonnes air                  0.000021  0.000051   
HFCs                     tonnes air                  0.000000  0.000000   
PFCs                     tonnes air                  0.000000  0.000000   
...                                                       ...       ...   
Ni                       tonnes soil                 0.000000  0.000000   
C                        tonnes air                  0.000000  0.000000   
other emissions          tonnes undef                0.018467  0.093876   
Carbon dioxide, biogenic tonnes air                  0.000174  0.000465   
Other emissions nec      tonnes air                  0.000000  0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air                 0.022910   
N2O                      tonnes air                 0.000154   
CH4                      tonnes air                 0.000003   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef               0.000443   
Carbon dioxide, biogenic tonnes air                 0.000029   
Other emissions nec      tonnes air                 0.000000   

region                                                                         \
sector                                Plant-based fibers Crops nec     Cattle   
Carbon dioxide, fossil   tonnes air             0.195246  2.327185   2.518185   
N2O                      tonnes air             0.001501  0.001340   0.066299   
CH4                      tonnes air             0.000016  0.000240   1.433311   
HFCs                     tonnes air             0.000000  0.000000   0.000000   
PFCs                     tonnes air             0.000000  0.000000   0.000000   
...                                                  ...       ...        ...   
Ni                       tonnes soil            0.000000  0.000000   0.000000   
C                        tonnes air             0.000000  0.000000   0.000000   
other emissions          tonnes undef           0.138161  0.293314  55.700957   
Carbon dioxide, biogenic tonnes air             0.000555  0.002586   0.000375   
Other emissions nec      tonnes air             0.000000  0.000000   0.000000   

region                                           ...  \
sector                                  

This are the UNITARY emissions per activity and per functional unit (F/X*L). Called M unitary emissions
Their column are like D_cba if Y is zeros with only a one

In [22]:
exio3318.emissions.M
#exio3318.emissions.M.xs("Precious metals", level="sector", axis=1)[0:3]
#exio3318.emissions.M.loc[('Carbon dioxide, fossil','tonnes','air'),("ZA","Precious metals")]


region                                        AU                              \
sector                                Paddy rice     Wheat Cereal grains nec   
Carbon dioxide, fossil   tonnes air     0.544363  0.195043          0.146194   
N2O                      tonnes air     0.000114  0.000503          0.000484   
CH4                      tonnes air     0.045244  0.000621          0.000517   
HFCs                     tonnes air     0.000000  0.000000          0.000000   
PFCs                     tonnes air     0.000000  0.000000          0.000000   
...                                          ...       ...               ...   
Ni                       tonnes soil    0.000000  0.000000          0.000000   
C                        tonnes air     0.000000  0.000000          0.000000   
other emissions          tonnes undef   0.003354  0.004409          0.006216   
Carbon dioxide, biogenic tonnes air     0.004347  0.003381          0.003109   
Other emissions nec      tonnes air     0.000000  0.000000          0.000000   

region                                                                   \
sector                                Vegetables, fruit, nuts Oil seeds   
Carbon dioxide, fossil   tonnes air                  1.030705  0.598524   
N2O                      tonnes air                  0.003039  0.001299   
CH4                      tonnes air                  0.004082  0.002646   
HFCs                     tonnes air                  0.000000  0.000000   
PFCs                     tonnes air                  0.000000  0.000000   
...                                                       ...       ...   
Ni                       tonnes soil                 0.000000  0.000000   
C                        tonnes air                  0.000000  0.000000   
other emissions          tonnes undef                0.039887  0.106325   
Carbon dioxide, biogenic tonnes air                  0.027842  0.015650   
Other emissions nec      tonnes air                  0.000000  0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air                 0.065622   
N2O                      tonnes air                 0.000190   
CH4                      tonnes air                 0.000217   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef               0.001714   
Carbon dioxide, biogenic tonnes air                 0.001554   
Other emissions nec      tonnes air                 0.000000   

region                                                                         \
sector                                Plant-based fibers Crops nec     Cattle   
Carbon dioxide, fossil   tonnes air             0.629466  7.255131   4.707365   
N2O                      tonnes air            -0.000096  0.002423   0.071714   
CH4                      tonnes air             0.002732  0.045038   1.474077   
HFCs                     tonnes air             0.000000  0.000000   0.000000   
PFCs                     tonnes air             0.000000  0.000000   0.000000   
...                                                  ...       ...        ...   
Ni                       tonnes soil            0.000000  0.000000   0.000000   
C                        tonnes air             0.000000  0.000000   0.000000   
other emissions          tonnes undef           0.014743  0.739212  58.410429   
Carbon dioxide, biogenic tonnes air             0.036358  0.627163   3.876396   
Other emissions nec      tonnes air             0.000000  0.000000   0.000000   

region                                           ...  \
sector                                  

This is the value we are looking for is M, but no the matrix multiplication (which is a sum), is a element-by-element information to keep all information needed. Impact_wanted is emmissions per functional unit of exiobase

In [23]:
country=input('Which country are you looking for?) ISO-2 code')
product=input('Which activity are you looking for? Exiobase product name')
impact_wanted=exio3318.emissions.S*exio3318.L.loc[:,(country,product)]
impact_wanted=impact_wanted.T
impact_wanted

Carbon dioxide, fossil  \
                                                                         tonnes   
                                                                            air   
region sector                                                                     
AU     Paddy rice                                                      0.000014   
       Wheat                                                           0.000651   
       Cereal grains nec                                               0.000276   
       Vegetables, fruit, nuts                                         0.000055   
       Oil seeds                                                      -0.000055   
...                                                                         ...   
WM     Membership organisation services n.e.c. (91)                    0.001029   
       Recreational, cultural and sporting services (92)               0.000895   
       Other services (93)                                             0.000271   
       Private households with employed persons (95)                   0.000000   
       Extra-territorial organizations and bodies                      0.000000   

                                                                   N2O  \
                                                                tonnes   
                                                                   air   
region sector                                                            
AU     Paddy rice                                         1.663328e-08   
       Wheat                                              3.178325e-06   
       Cereal grains nec                                  2.184243e-06   
       Vegetables, fruit, nuts                            1.885388e-06   
       Oil seeds                                         -3.093762e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                   CH4   HFCs  \
                                                                tonnes tonnes   
                                                                   air    air   
region sector                                                                   
AU     Paddy rice                                         1.000737e-05    0.0   
       Wheat                                              5.483440e-08    0.0   
       Cereal grains nec                                  7.751717e-08    0.0   
       Vegetables, fruit, nuts                            1.500764e-08    0.0   
       Oil seeds                                         -1.372622e-08   -0.0   
...                                                                ...    ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00    0.0   
       Recreational, cultural and sporting services (92)  0.000000e+00    0.0   
       Other services (93)                                0.000000e+00    0.0   
       Private households with employed persons (95)      0.000000e+00    0.0   
       Extra-territorial organizations and bodies         0.000000e+00    0.0   

                                                           PFCs    SF6  \
                                                         tonnes tonnes   
                                                            air    air   
region sector                                                            
AU     Paddy rice                                           0.0    0.0   
       Wheat                                                0.0    0.0   
       Cereal grains nec                

In [24]:
impact_wanted['Carbon dioxide, fossil', 'tonnes', 'air'].nlargest(10) #The most contaminant activities in term of CO2

region  sector                                                                                                                
ZA      Electricity by coal                                                                                                       3525.921263
        Precious metals                                                                                                            519.957020
        Precious metal ores and concentrates                                                                                       504.820668
WF      Electricity by petroleum and other oil derivatives                                                                          43.239257
ZA       Refined Petroleum                                                                                                          22.049573
        Air transport services (62)                                                                                                 18.200058
RU      Steam and hot

In [25]:
impact_wanted.to_csv('ES.txt', sep='\t', index=True, header=True)

# Calculation for BW2.5

To join all impact per country (To have a faster code)

In [26]:
impact_per_country=impact_wanted.groupby('region').sum()
impact_per_country

,"Carbon dioxide, fossil",N2O,CH4,HFCs,PFCs,SF6,NOX,SOx,NH3,NMVOC,...,Cu,Zn,Pb,Hg,Cr,Ni,C,other emissions,"Carbon dioxide, biogenic",Other emissions nec
,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,...,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes,tonnes
,air,air,air,air,air,air,air,air,air,air,...,soil,soil,soil,soil,soil,soil,air,undef,air,air
region,,,,,,,,,,,,,,,,,,,,,
AT,0.057777,4.410499e-06,0.000079,0.0,0.0,0.0,0.000168,0.000037,0.000034,0.000069,...,0.0,1.236528e-09,1.015058e-09,0.0,0.0,0.0,0.0,0.002539,0.007961,0.0
AU,0.395030,1.195260e-04,0.003691,0.0,0.0,0.0,0.001281,0.001058,0.000901,0.001488,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.072899,0.016430,0.0
BE,0.310266,2.099168e-05,0.000266,0.0,0.0,0.0,0.000783,0.000301,0.000136,0.000314,...,0.0,2.561943e-07,3.438759e-08,0.0,0.0,0.0,0.0,0.016261,0.030676,0.0
BG,0.175692,8.057685e-06,0.000213,0.0,0.0,0.0,0.000487,0.001533,0.000019,0.001880,...,0.0,2.990010e-09,4.002222e-10,0.0,0.0,0.0,0.0,0.009040,0.005664,0.0
BR,0.076045,3.432742e-05,0.006323,0.0,0.0,0.0,0.000385,0.000798,0.000302,0.001194,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.015989,0.033799,0.0
CA,0.617650,4.811616e-05,0.013911,0.0,0.0,0.0,0.001992,0.001205,0.000282,0.001477,...,0.0,2.427211e-10,2.046149e-10,0.0,0.0,0.0,0.0,0.014260,0.026384,0.0
CH,0.162838,6.569836e-06,0.000082,0.0,0.0,0.0,0.000426,0.000112,0.000034,0.000342,...,0.0,2.288236e-08,1.244297e-08,0.0,0.0,0.0,0.0,0.006402,0.007057,0.0


In [27]:
impact_per_country.iloc[0,0]

0.057777233988237225

To install brightway 2.5 in jupyter (only if it is necessary)

In [28]:
import sys
print(sys.executable) #to know where is python
#!{sys.executable} -m pip install brightway25 -y
#!{sys.executable} -m pip install bw2io -y
#!{sys.executable} -m pip uninstall pypardiso -y #The code does not work with pypardiso installed. Why? I do not know

c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe


Libraries for brightway2.5

In [29]:
import brightway25 as bw
import bw2io as bi
import bw2data as bd
import bw2analyzer as ba
import bw2calc as bc
import matrix_utils as mu
import bw_processing as bp
print(bc.__version__)
print(bd.__version__)
import numpy as np

if any("biosphere3" in db for db in bd.databases): #to confirm biosphere is present
    print('Biosphere3 is already present in the project.')
else:
    bi.bw2setup()

#bd.projects.migrate_project_25() #only for the first time changing brigthway2 to 2.5

c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\brightway25\__init__.py:9: UserWarning: The Brightway25 package doesn't provide anything; import the constituent libraries separately
  warnings.warn("The Brightway25 package doesn't provide anything; import the constituent libraries separately")


11:47:48+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.
2.0.2
(4, 4, 4)
Biosphere3 is already present in the project.


To confirm biosphere is well installed

In [30]:
print(len(bd.methods)) #This number is not 1, if it is 1; try to run in an other kernel
if len(bd.methods)==1:
    print('You have a problem in brightway installation, only one mtehod have been added. Open another kernel and start again')

762


This code is based on EF v3.1

In [31]:
method1='EF v3.1' #Change if you want another one
#method2=input ('Which category of EF v3.1 would you like to use? (part 1) Write exactly as in brightway between upper commas')
method2='climate change'
method3='global warming potential (GWP100)' #Change if you want another one
#method3=input ('Which category of EF v3.1 would you like to use? (part 2) Write exactly as in brightway between upper commas')
method_raw=(method1,method2,method3)
method_chosen=tuple(x.strip().replace("'", "").replace('"', '') for x in method_raw) #to put in correct format (it has "")
print(method_chosen) #to confirm it is correct

('EF v3.1', 'climate change', 'global warming potential (GWP100)')


To calculate equivalent CO2, using brightway

In [32]:
co2_flows = [act for act in bd.Database('biosphere3') if 'carbon dioxide, fossil' in act['name'].lower()] #To select the key for emissions
co2_key = co2_flows[0].key 
n2o_flows = [act for act in bd.Database('biosphere3') if 'dinitrogen monoxide' in act['name'].lower()]
n2o_key = n2o_flows[0].key 
ch4_flows = [act for act in bd.Database('biosphere3') if 'methane, fossil' in act['name'].lower()]
ch4_key = ch4_flows[0].key 


In [33]:
M_eq=[]
flag=0
for i, region in impact_per_country.iterrows(): #Creating a proccess with emissions
    flag=flag+1
    print(i)
    process_data={'name':f"process_{flag}" ,
            'exchanges': [
                {'input': ("Emissions", f"process_{flag}"), 'amount': 1, 'type': 'production'},
                {'input': co2_key, 'amount': impact_per_country.loc[i,('Carbon dioxide, fossil','tonnes','air')]*1000, 'type': 'biosphere', 'location': i},#to add the emissions of each activity
                {'input': n2o_key, 'amount': impact_per_country.loc[i,('N2O','tonnes','air')]*1000, 'type': 'biosphere', 'location': i},
                {'input': ch4_key, 'amount': impact_per_country.loc[i,('CH4','tonnes','air')]*1000, 'type': 'biosphere', 'location': i}
            ]
        }
    print(process_data)
    bd.Database("Emissions").write({("Emissions", f"process_{flag}"): process_data}) #To add in database
    demand_i={("Emissions", f"process_{flag}"):1} #1 as functional unit
    value=bc.lca.LCA(demand_i,method_chosen) 
    value.lci(factorize=True) #with factorization false doesn't work
    value.lcia()


    M_eq.append(value.score/1000) #to store the equivalent emissions in M_eq_val in tonnes

M_eq=pd.DataFrame(M_eq)
M_eq.index=impact_per_country.index
M_eq.columns=["CO2-eq"] #To name the columns

AT
{'name': 'process_1', 'exchanges': [{'input': ('Emissions', 'process_1'), 'amount': 1, 'type': 'production'}, {'input': ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 'amount': 57.77723398823723, 'type': 'biosphere', 'location': 'AT'}, {'input': ('biosphere3', '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb'), 'amount': 0.004410499086171631, 'type': 'biosphere', 'location': 'AT'}, {'input': ('biosphere3', '5f7aad3d-566c-4d0d-ad59-e765f971aa0f'), 'amount': 0.07927432620127836, 'type': 'biosphere', 'location': 'AT'}]}
11:47:54+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 335.12it/s]


11:47:54+0200 [info     ] Vacuuming database            
AU
{'name': 'process_2', 'exchanges': [{'input': ('Emissions', 'process_2'), 'amount': 1, 'type': 'production'}, {'input': ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 'amount': 395.0300720872022, 'type': 'biosphere', 'location': 'AU'}, {'input': ('biosphere3', '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb'), 'amount': 0.11952600114572136, 'type': 'biosphere', 'location': 'AU'}, {'input': ('biosphere3', '5f7aad3d-566c-4d0d-ad59-e765f971aa0f'), 'amount': 3.6913543646804685, 'type': 'biosphere', 'location': 'AU'}]}
11:47:55+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<?, ?it/s]


11:47:55+0200 [info     ] Vacuuming database            
BE
{'name': 'process_3', 'exchanges': [{'input': ('Emissions', 'process_3'), 'amount': 1, 'type': 'production'}, {'input': ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 'amount': 310.2658828169094, 'type': 'biosphere', 'location': 'BE'}, {'input': ('biosphere3', '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb'), 'amount': 0.020991683181195004, 'type': 'biosphere', 'location': 'BE'}, {'input': ('biosphere3', '5f7aad3d-566c-4d0d-ad59-e765f971aa0f'), 'amount': 0.2655796264925214, 'type': 'biosphere', 'location': 'BE'}]}
11:47:56+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


...

11:48:19+0200 [info     ] Vacuuming database            
WM
{'name': 'process_47', 'exchanges': [{'input': ('Emissions', 'process_47'), 'amount': 1, 'type': 'production'}, {'input': ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 'amount': 2458.587184513766, 'type': 'biosphere', 'location': 'WM'}, {'input': ('biosphere3', '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb'), 'amount': 0.16143035959701174, 'type': 'biosphere', 'location': 'WM'}, {'input': ('biosphere3', '5f7aad3d-566c-4d0d-ad59-e765f971aa0f'), 'amount': 315.2502606407043, 'type': 'biosphere', 'location': 'WM'}]}
11:48:19+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<?, ?it/s]


11:48:19+0200 [info     ] Vacuuming database            
ZA
{'name': 'process_48', 'exchanges': [{'input': ('Emissions', 'process_48'), 'amount': 1, 'type': 'production'}, {'input': ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 'amount': 4606897.776412499, 'type': 'biosphere', 'location': 'ZA'}, {'input': ('biosphere3', '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb'), 'amount': 72.5072125167367, 'type': 'biosphere', 'location': 'ZA'}, {'input': ('biosphere3', '5f7aad3d-566c-4d0d-ad59-e765f971aa0f'), 'amount': 14681.05400228077, 'type': 'biosphere', 'location': 'ZA'}]}
11:48:20+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<?, ?it/s]


11:48:20+0200 [info     ] Vacuuming database            


In [34]:
M_eq

,CO2-eq
region,
AT,0.061344
AU,0.537663
BE,0.323911
BG,0.184233
BR,0.273852
CA,1.045325
CH,0.167077
CN,3.084733
CY,0.015135


# Use of ecoinvent to update and calculate and specific impact

In [35]:
imp_total=M_eq.sum()#To calculate the percentage of impact per country
print(imp_total)
M_perc=M_eq/abs(imp_total)

CO2-eq    5252.414364
dtype: float64


In [36]:
answer=input("Do you want to update and make more specific the regionalised impact? Yo need to add information of ecoinvent or similar. Put y for yes, n for no").strip().lower()

if answer=="y":
    #Updating with ecoinvent
    value_LCA_conv=input ('Which is the update and specific impact of this activity? Global or regional obtained from ecoinvent or other database (decimals with point): ') #To obtain an update and specific impact
    value_LCA_conv=float(value_LCA_conv) #Numbers are stored as string, to change it to numbers

    #Better regionalization
    impact_countries_mean=[]
    for i, region in M_eq.iterrows(): #Creating a proccess with emissions
        print(i)
        imp_one_country={'name':f"impact_{i}",
                'exchanges': [
                    {'input': ("Emissions", f"impact_{i}"), 'amount': 1, 'type': 'production'},
                    {'input': co2_key, 'amount': exio3318.emissions.M.loc[('Carbon dioxide, fossil','tonnes','air'),(i,product)]*1000, 'type': 'biosphere', 'location': i},#to add the emissions of each activity
                    {'input': n2o_key, 'amount': exio3318.emissions.M.loc[('N2O','tonnes','air'),(i,product)]*1000, 'type': 'biosphere', 'location': i},
                    {'input': ch4_key, 'amount': exio3318.emissions.M.loc[('CH4','tonnes','air'),(i,product)]*1000, 'type': 'biosphere', 'location': i}
                ]
            }
        print(imp_one_country)
        bd.Database("Emissions").write({("Emissions", f"impact_{i}"): imp_one_country}) #To add in database
        demand_i={("Emissions", f"impact_{i}"):1} #1 as functional unit
        value=bc.lca.LCA(demand_i,method_chosen) 
        value.lci(factorize=True) #with factorization false doesn't work
        value.lcia()
        impact_countries_mean.append(value.score/1000) #to store the equivalent emissions in impact_countries_mean in tonnes

    impact_countries_mean=pd.DataFrame(impact_countries_mean)
    impact_countries_mean.index=M_eq.index
    impact_countries_mean.columns=["CO2-eq"] #To name the columns 
    imp_this_country=impact_countries_mean.loc[country]
    imp_global=impact_countries_mean.mean()
    param_k=imp_this_country.iloc[0]/abs(imp_global.iloc[0]) #k value
    P_eq=M_perc*value_LCA_conv*param_k  #To calculate updated and specific impact per country
    P_eq.index=M_eq.index #To add headings in P_eq (The rows)
    print(P_eq)
elif answer=="n":
    P_eq=M_perc*imp_total  #To calculate updated and specific impact per country 
else:
    print("Error please write y or n")

# Social impact

F social (Extensions, total QALY per activity)

In [ ]:
F_social_raw=pd.read_excel(exiobase_folder + "SF update 2022_QALY 2019 by skill level and industry using Exiobase 2011.xlsx", #This data is QALY of intermediate production
                      sheet_name="VA_act and social footprint", #You should have this name of file, if not change it.
                      index_col=[0,1,2,3],
                      header=[0,1,2,3])
F_social_raw.columns=exio3318.emissions.F.columns 
F_social_reindexed=F_social_raw.droplevel([0,1], axis=0).copy() #To remove useless columns     
F_social_reindexed
exio3318.emissions.Fs=F_social_reindexed
exio3318.emissions.Fs

Definition of vector "S" but with social information

In [39]:
total_qaly=exio3318.emissions.Fs.loc[('Sum of non-production-specific lost wellbeing by industry'),:].T.astype(float)
people=exio3318.emissions.Fs.loc[('Country population 2011'),:].T.astype(float)
x=exio3318.x.copy().astype(float)
people.columns=total_qaly.columns
x.columns=total_qaly.columns
people_qaly=total_qaly.div(people)
unit_qaly=people_qaly.div(x)

unit_qaly=unit_qaly.replace([np.inf, -np.inf, np.nan], 0)
exio3318.emissions.S_social=unit_qaly.T
exio3318.emissions.S_social

region            AU                                                          \
sector    Paddy rice         Wheat Cereal grains nec Vegetables, fruit, nuts   
QALY    3.633281e-11  5.928685e-11      2.282924e-11            3.389475e-10   

region                                                                        \
sector     Oil seeds Sugar cane, sugar beet Plant-based fibers     Crops nec   
QALY    2.166242e-10           1.240599e-11       1.707100e-09  2.286336e-09   

region                              ...                            WM  \
sector        Cattle          Pigs  ... Paper for treatment: landfill   
QALY    9.415687e-10  1.170267e-09  ...                  8.777310e-12   

region                                        \
sector Plastic waste for treatment: landfill   
QALY                            7.070085e-12   

region                                                      \
sector Inert/metal/hazardous waste for treatment: landfill   
QALY                                         2.750934e-12    

region                                         \
sector Textiles waste for treatment: landfill   
QALY                             1.257321e-11   

region                                     \
sector Wood waste for treatment: landfill   
QALY                         6.851666e-11   

region                                               \
sector Membership organisation services n.e.c. (91)   
QALY                                   1.219877e-07   

region                                                                        \
sector Recreational, cultural and sporting services (92) Other services (93)   
QALY                                        9.399257e-08        8.172186e-08   

region                                                \
sector Private households with employed persons (95)   
QALY                                    2.037555e-07   

region                                             
sector Extra-territorial organizations and bodies  
QALY                                          0.0  

[1 rows x 7872 columns]

Calculate the impact of one category (vector M with social information with all information, element by element)

In [40]:
impact_wanted_social=exio3318.emissions.S_social*exio3318.L.loc[:,(country,product)]
impact_wanted_social=impact_wanted_social.T
impact_wanted_social

P_social=impact_wanted_social.groupby('region').sum()
P_social

,QALY
region,
AT,1.995944e-10
AU,9.102649e-11
BE,9.924699e-10
BG,8.440992e-10
BR,3.042392e-11
CA,7.739856e-09
CH,4.599215e-10
CN,1.649286e-10
CY,9.187972e-10


# Economic impact

F economic (Extensions, total MEUR per activity, sum of columns of IO economic)

In [ ]:
F_eco_raw=pd.read_excel(exiobase_folder + "MR_HIOT_2011_total_cost.xlsx", #This data is emissions of intermediate production
                      sheet_name="Total_cost", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_eco_raw.columns=Z_reindexed.columns
exio3318.emissions.F_e=F_eco_raw
exio3318.emissions.F_e

In [41]:
exio3318.L.loc[:,(country,product)].nsmallest(10) 

region  sector                                                                               
ZA      Cement, lime and plaster                                                                -29.742639
        Chemical and fertilizer minerals, salt and other mining and quarrying products n.e.c.    -3.012501
WA      Cement, lime and plaster                                                                 -1.844396
WF      Cement, lime and plaster                                                                 -1.820040
WM      Cement, lime and plaster                                                                 -1.432612
WF      Sand and clay                                                                            -1.044101
WM      Sand and clay                                                                            -0.897038
WA      Sand and clay                                                                            -0.529832
JP      Basic iron and steel and of ferro-alloys a

In [44]:
total_cost=exio3318.emissions.F_e.loc[('Total cost'),:].T.astype(float)
x=exio3318.x.copy().astype(float)
x.columns=total_cost.columns
unit_cost=total_cost.div(x)

unit_cost=unit_cost.replace([np.inf, -np.inf, np.nan], 0)
exio3318.emissions.S_eco=unit_cost.T
exio3318.emissions.S_eco

region                    AU                              \
sector            Paddy rice     Wheat Cereal grains nec   
MEUR Sum_colum_IO   0.000178  0.000149          0.000152   

region                                                                      \
sector            Vegetables, fruit, nuts Oil seeds Sugar cane, sugar beet   
MEUR Sum_colum_IO                 0.00177  0.003114               0.000068   

region                                                              ...  \
sector            Plant-based fibers Crops nec    Cattle      Pigs  ...   
MEUR Sum_colum_IO           0.005935  0.083199  0.004275  0.001376  ...   

region                                       WM  \
sector            Paper for treatment: landfill   
MEUR Sum_colum_IO                      0.002556   

region                                                   \
sector            Plastic waste for treatment: landfill   
MEUR Sum_colum_IO                              0.012398   

region                                                                 \
sector            Inert/metal/hazardous waste for treatment: landfill   
MEUR Sum_colum_IO                                           0.000005    

region                                                    \
sector            Textiles waste for treatment: landfill   
MEUR Sum_colum_IO                               0.000835   

region                                                \
sector            Wood waste for treatment: landfill   
MEUR Sum_colum_IO                           0.009572   

region                                                          \
sector            Membership organisation services n.e.c. (91)   
MEUR Sum_colum_IO                                     0.639471   

region                                                               \
sector            Recreational, cultural and sporting services (92)   
MEUR Sum_colum_IO                                          0.547422   

region                                 \
sector            Other services (93)   
MEUR Sum_colum_IO            0.486492   

region                                                           \
sector            Private households with employed persons (95)   
MEUR Sum_colum_IO                                      0.404822   

region                                                        
sector            Extra-territorial organizations and bodies  
MEUR Sum_colum_IO                                        0.0  

[1 rows x 7872 columns]

Calculate the impact of one category (vector M with economic information with all information, element by element)

In [45]:
impact_wanted_eco=exio3318.emissions.S_eco*exio3318.L.loc[:,(country,product)]
impact_wanted_eco=impact_wanted_eco.T
impact_wanted_eco

P_eco=impact_wanted_eco.groupby('region').sum()
P_eco

,MEUR
,Sum_colum_IO
region,
AT,0.000212
AU,0.000530
BE,0.001329
BG,0.000298
BR,0.000234
CA,0.001680
CH,0.002758
CN,0.003483


# To save the values as TXT (change the name of file if you want)

In [46]:
answer2=input("Do you want to save in txt files value of enivornmental, social and economic impact? Put y for yes, n for no").strip().lower()

if answer2=="y":
    impact_wanted.to_csv('Emissions_products.txt', sep='\t', index=True, header=True)
    P_eq.to_csv('Environmental_impact.txt', sep='\t', index=True, header=True)
    P_social.to_csv('Social_impact.txt', sep='\t', index=True, header=True)
    P_eco.to_csv('Cost_impact.txt', sep='\t', index=True, header=True)
    #np.savetxt('M_social.txt',M_social, fmt="%s") #To save the M_social
    #np.savetxt('M_economic.txt',M_eco, fmt="%s") #To save the M_economic
    print("txt files are in the same folder of this file")
elif answer2=="n":
    print("Not saved")
else:
    print("Error please write y or n")

Not saved


# To do the maps of impacts 

To install nbformat in jupyter (only if it is necessary)

In [ ]:
#import sys
#print(sys.executable) #to know where is python
#!{sys.executable} -m pip install --upgrade nbformat

To activate plotly and read countries in format ISO 3 (needed by plotly). Install plotly if needed

In [47]:
#import sys
#print(sys.executable) #to know where is python
#!{sys.executable} -m pip install plotly 
import plotly.express as px
countries_alpha3=pd.read_csv('countries_alpha3.csv', sep=';') #To create the dataframe

To do the maps of environmental impacts (I don't why these lines sometimes do not work, re-run the code if maps are not painted)

In [48]:
import nbformat
P_eq_df=P_eq.reset_index()
P_eq_df.columns=["EXIOBASE","Impact"] #To name the columns P_eq_df

P_eq_join=pd.merge(countries_alpha3,P_eq_df,how="outer") #Join two dataframe in one 
    
P_eq_join=P_eq_join.dropna(axis=1) #To eliminate empty columns (something should not work correctly)
P_eq_join['Impact']=P_eq_join['Impact'].astype(float) 

fig= px.choropleth(P_eq_join,locations='ISO3',color='Impact',color_continuous_scale=px.colors.sequential.Greens) #To paint the map, copy from internet

fig.show() #To show the map

To do the maps of social impacts (I don't why these lines sometimes do not work, re-run the code if maps are not painted)

In [49]:
import nbformat
P_social_df=P_social.reset_index()
P_social_df.columns=["EXIOBASE","Impact"] #To name the columns P_social_df

P_social_join=pd.merge(countries_alpha3,P_social_df,how="outer") #Join two dataframe in one 
    
P_social_join=P_social_join.dropna(axis=1) #To eliminate empty columns (something should not work correctly)
P_social_join['Impact']=P_social_join['Impact'].astype(float) 

fig_s=px.choropleth(P_social_join,locations='ISO3',color='Impact',color_continuous_scale=px.colors.sequential.Magenta) #To paint the map, copy from internet

fig_s.show() #To show the map

To do the maps of economic impacts (I don't why these lines sometimes do not work, re-run the code if maps are not painted)

In [50]:
import nbformat
P_eco_df=P_eco.reset_index()
P_eco_df.columns=["EXIOBASE","Impact"] #To name the columns P_eco_df

P_eco_join=pd.merge(countries_alpha3,P_eco_df,how="outer") #Join two dataframe in one 
    
P_eco_join=P_eco_join.dropna(axis=1) #To eliminate empty columns (something should not work correctly)
P_eco_join['Impact']=P_eco_join['Impact'].astype(float) 

fig_e=px.choropleth(P_eco_join,locations='ISO3',color='Impact',color_continuous_scale=px.colors.sequential.Oranges) #To paint the map, copy from internet

fig_e.show() #To show the map